In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at010-agent-chat"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='ea03ec4f-37a6-41bc-b020-c7b63e1a5579', created_at=datetime.datetime(2025, 8, 19, 17, 23, 54, 432030, tzinfo=TzInfo(UTC)), name='07174a88-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 8, 19, 17, 23, 54, 432030, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='c6241128-0350-434e-b196-2d0fbbb7df9a', agent_id='a1abb90e-c673-4448-a4e2-841170568840', sequence_id=914, task_id='ea03ec4f-37a6-41bc-b020-c7b63e1a5579', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 19, 17, 23, 54, 466983, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,
)

╭───────────────────────── USER [08/19/2025 17:23:54] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/19/2025 17:24:01] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Fetching latest AI news                                                      │
│                                                                              │
│ The user wants to know the latest news about AI and AI startups. I need to   │
│ gather this information today, so I’ll use the web_search tool to find       │
│ recent articles. I plan to search for "AI news August 2025," but I'll expand │
│ it to include keywords about funding, layoffs, regulations, models, and      │
│ chips. This way, I can provide a comprehensive update. Let's see what the    │
│ search turns up!                                                             │
╰──────────────────────────────────────────────────────────────────────────────╯

⠋ 🔄 Agent responding...Unknown TaskMessageUpdate type: None
⠏ 🔄 Agent responding...Unknown TaskMessageUpdate type: None
⠋ 🔄 Agent responding...Unknown TaskMessageUpdate type: None
⠦ 🔄 Agent responding...Unknown TaskMessageUpdate type: None


╭──────────────────────── AGENT [08/19/2025 17:24:30] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Summarizing recent news                                                      │
│                                                                              │
│ The web search provided text about recent news from August 19, 2025,         │
│ highlighting key items. I need to ensure the details about model releases,   │
│ like OpenAI and its funding, are presented accurately, as some might be      │
│ misleading. We should summarize funding rounds, partnerships, and            │
│ acquisitions, but there was no mention of layoffs in the search results. I   │
│ could also look into AI regulation developments, specifically for the EU AI  │
│ Act and US executive actions. Let's conduct another search for those.        │
╰──────────────────────────────────────────────────────────────────────────────╯

⠋ 🔄 Agent responding...Unknown TaskMessageUpdate type: None
⠴ 🔄 Agent responding...Unknown TaskMessageUpdate type: None


╭──────────────────────── AGENT [08/19/2025 17:24:51] ─────────────────────────╮
│ Here’s a concise roundup of the biggest AI / AI-startup stories as of today  │
│ (19 Aug 2025), with the main takeaways and where they matter most. Tell me   │
│ which topics you want deeper coverage on (funding, models & chips,           │
│ regulation, M&A, robotics, security, or job-market impact).                  │
│                                                                              │
│ Top funding and startup moves                                                │
│                                                                              │
│  • OpenAI — reported a large private funding round (~$8.3B) at a very high   │
│    valuation (reports flagged ~ $300B). Funds aimed at scaling model         │
│    development and infrastructure. (news reports Aug 2025)                   │
│  • Apptronik — big raise (~$350M Series A) to scale humanoid-robot           │
│    production and commercial deployments in logistics/industrial settings.   │
│  • FuriosaAI — raised a growth round (reported ~$125M) to expand production  │
│    of its AI inference chips for data centers.                               │
│  • Noma Security, Nudge and other niche startups — significant rounds        │
│    (cybersecurity for models, brain-tech / non‑invasive neural interfaces)   │
│    reflecting continued investor interest in specialized AI applications.    │
│                                                                              │
│ Model, hardware and product developments                                     │
│                                                                              │
│  • Continued focus on inference efficiency and custom accelerators: startups │
│    and chipmakers (like FuriosaAI) are raising to bring new inference ASICs  │
│    to market as data-center demand grows.                                    │
│  • Robotics commercialization: companies such as Apptronik are moving from   │
│    prototypes to production planning, with strategic investors showing       │
│    interest.                                                                 │
│  • AI safety/productization tooling: rising investment in model-security,    │
│    governance, and monitoring platforms (to protect IP, prevent data leaks,  │
│    and detect model attacks).                                                │
│                                                                              │
│ Partnerships, M&A and strategic moves                                        │
│                                                                              │
│  • Strategic partnerships between automotive/commercial vehicle groups and   │
│    autonomy/software specialists continue to accelerate (e.g.,               │
│    software-defined truck and autonomy platform deals).                      │
│  • Some acquisitions and deals in adjacent Web3/DeFi sectors show incumbents │
│    reshaping infrastructure — illustrating cross-industry consolidation that │
│    can affect AI startups’ access to capital and customers.                  │
│                                                                              │
│ Regulation & policy (major developments)                                     │
│                                                                              │
│  • EU AI Act enforcement is in active phases — the EU’s risk-based rules and │
│    enforcement mechanisms are being implemented and starting to bite for     │
│    high‑risk systems. Firms targeting EU customers must align documentation, │
│    risk assessments, and compliance processes.                               │
│  • U.S. policy direction shifted in 2025 with new executive-level AI         │
│    initiatives emphasizing faster deployment, infrastructure incentives, and │
│    lighter federal regulation (reports indicate debate over balance between  │
│    innovation and consumer pr

Streaming timed out after 60 seconds - returning collected messages
